In [ ]:
import os
import json
from ape import networks
import pandas as pd
import matplotlib.pyplot as plt
from CreditGuild.controllers.GuildTokenHolders import fetch_holders
from CreditGuild.controllers.ActiveLendingTerms import fetch_terms_and_debt, fetch_users_stakes


with open('CreditGuild/addresses.json', 'r') as file:
    addresses = json.load(file)

guild_token_address = addresses.get('token_address')
top_addresses = addresses.get('top_addresses')
stake_minters = addresses.get('stake_minters')

top_addresses_df = pd.DataFrame(top_addresses)


In [ ]:
holders_data = None
holders_csv_file_path = "data/credit_guild_guild_token_holders.csv"
if os.path.exists(holders_csv_file_path ):
    holders_data = pd.read_csv(holders_csv_file_path, header=0)

with networks.parse_network_choice("arbitrum:mainnet:infura") as provider:
    if holders_data is None:
        str_addresses = [address["address"] for address in top_addresses]
        holders_data = fetch_holders(str_addresses, guild_token_address)
        holders_data.to_csv(holders_csv_file_path, index=False, header=True)

holder_balances = pd.merge(holders_data, top_addresses_df, on="address", how="inner")

In [ ]:
terms_data = None
terms_debt_csv_file_path = 'data/credit_guild_terms_debt.csv'
if os.path.exists(terms_debt_csv_file_path):
    terms_data = pd.read_csv(terms_debt_csv_file_path)

with networks.parse_network_choice("arbitrum:mainnet:infura") as provider:
    terms_data = terms_data if not terms_data is None else fetch_terms_and_debt(guild_token_address)
    terms_data.to_csv(terms_debt_csv_file_path, index=False, header=True)

In [ ]:
user_stakes_csv_file_path = 'data/credit_guild_user_stakes.csv'

merged_staking = pd.merge(terms_data, pd.DataFrame(stake_minters), on="creditToken", how="inner")

users_stakes = None
if os.path.exists(user_stakes_csv_file_path):
    users_stakes = pd.read_csv(user_stakes_csv_file_path, header=0)

with networks.parse_network_choice("arbitrum:mainnet:infura") as provider:
    if users_stakes is None:
        users_stakes = fetch_users_stakes(top_addresses_df, merged_staking, slow=True)
    users_stakes.to_csv(user_stakes_csv_file_path, index=False, header=True)

In [ ]:
holder_balances['balance'] = pd.to_numeric(holder_balances['balance'], errors='coerce')
holder_balances = holder_balances.sort_values(by='balance', ascending=False)
holder_balances.plot(kind='bar', x='nickname', y='balance', legend=True)

# Add labels and title
plt.xlabel('Name')
plt.ylabel('Value')
plt.title('Guild Token Top Holders')

# Display the plot
plt.show()